# Support for Quantum ESPRESSO in exatomic

In [1]:
import os
import exatomic
from exatomic import qe


def get_path(name):
    """Helper function for building paths"""
    return os.path.abspath(os.path.join(os.path.abspath(exatomic.__file__), "../static/qe/qe."+name+".bz2"))

In [2]:
#path = os.path.abspath(os.path.join(os.path.abspath(exatomic.__file__), "../static/qe/qe.inp.bz2"))
symbols = qe.parse_symbols_from_input(get_path("inp"))

In [3]:
pos = qe.parse_xyz(get_path("pos"), symbols)
vel = qe.parse_xyz(get_path("vel"), symbols)
force = qe.parse_xyz(get_path("for"), symbols)

In [7]:
evp = qe.parse_evp(get_path("evp"), symbols)
evp['atom_count'] = len(symbols)

In [22]:
import pandas as pd
atom = pos
uni = exatomic.Universe(atom=atom, frame=evp)

In [23]:
len(uni)

10001

In [26]:
uni.atom.dtypes

x          float64
y          float64
z          float64
frame     category
symbol    category
dtype: object

In [27]:
%time uni.compute_atom_two()

CPU times: user 4.36 s, sys: 124 ms, total: 4.48 s
Wall time: 4.46 s
